<a href="https://colab.research.google.com/github/norhum/deep_learning/blob/main/wikipytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1)

x_train = torch.tensor([[73, 80, 75], [93, 88, 93], [89, 91, 90], [96, 98, 100], [73, 66, 70]]).float() #(5, 3)
y_train = torch.tensor([[152], [185], [180], [196], [142]]).float() #(5, 1)

lr = 1e-5
epoch = 1000

class MVLRM(nn.Module):
  def __init__(self, input_size, output_size):
    super().__init__()
    self.linear = nn.Linear(input_size, output_size)

  def forward(self, x):
    return self.linear(x)

model = MVLRM(x_train.shape[1], y_train.shape[1])

optimizer = torch.optim.SGD(model.parameters(), lr)

for i in range(epoch):
  output = model(x_train)
  loss = F.mse_loss(output, y_train)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if i % 100 == 0:
    print(f'Iter {i} :  {loss.item()}')



Iter 0 :  31667.59765625
Iter 100 :  0.22599251568317413
Iter 200 :  0.22391095757484436
Iter 300 :  0.22194059193134308
Iter 400 :  0.2200593650341034
Iter 500 :  0.21827054023742676
Iter 600 :  0.21657471358776093
Iter 700 :  0.21495017409324646
Iter 800 :  0.21341314911842346
Iter 900 :  0.21195237338542938


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1)

x_data = torch.tensor([[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]).float()
y_data = torch.tensor([[0], [0], [0], [1], [1], [1]]).float()

model = nn.Sequential(
    nn.Linear(x_data.shape[1], y_data.shape[1]),
    nn.Sigmoid()
)
# could make a class like below
class BinaryClassifier(nn.Module):
  def __init__(self, input_size, output_size):
    super().__init__()
    self.linear = nn.Linear(input_size, output_size)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    out = self.sigmoid(self.linear(x))
    return out

epoch = 100
lr = 1e-1

optimizer = torch.optim.AdamW(model.parameters(), lr)

for i in range(epoch):
  out = model(x_data)
  # loss = F.binary_cross_entropy(out, y_data)
  # loss = -(y_data * torch.log(out) + (1-y_data) * torch.log(1 - out)).mean()
  loss = 0
  for j in range(len(y_data)):
    if y_data[j, 0] == 1:
      loss += -torch.log(out[j, 0])
    else:
      loss += -torch.log(1-out[j, 0])
  loss /= len(y_data)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if i % 10 == 0:
    print(loss.item())

# from F.binary_cross_entropy function == loss which I derived from the wiki page == loss that I made from scratch
# 0.5397130846977234
# 0.41038432717323303
# 0.3289487063884735
# 0.2697530686855316
# 0.22443558275699615
# 0.1899040937423706
# 0.1634930819272995
# 0.14306499063968658
# 0.1269247978925705
# 0.11390752345323563

print(list(model.parameters()))

# 왜 BCE이 CE이랑 근본적으로 같은지 BinaryCE를 그냥 CE로 코딩으로 구현해보기

0.5397131443023682
0.41038432717323303
0.3289487063884735
0.2697530686855316
0.22443561255931854
0.1899040937423706
0.1634930819272995
0.14306500554084778
0.12692482769489288
0.11390754580497742
[Parameter containing:
tensor([[1.7394, 0.5980]], requires_grad=True), Parameter containing:
tensor([-7.1584], requires_grad=True)]


In [ ]:
# MLP로 XOR 문제 풀기 (BCE가 아닌 CE로 풀었음)
import torch
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1)
if device == 'cuda':
  torch.cuda.manual_seed_all(1)

X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.Tensor([0, 1, 1, 0]).long().to(device)
F = nn.ReLU()
lr = 5e-2
# 5e-1 doesn't work for ReLU and Tanh but does for Sigmoid. Why?

model = nn.Sequential(
    nn.Linear(2, 10), F,
    nn.Linear(10, 10), F,
    nn.Linear(10, 10), F,
    nn.Linear(10, 2), F
).to(device)

function = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr)

for i in range(1000):
  loss = function(model(X), Y)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if i % 100 == 0:
    print(loss.item())

with torch.no_grad():
  test = model(X)
  print(Y, test)
  a = torch.tensor([test[i,Y[i]] for i in range(len(Y))])
  b = a > 0.5
  accuracy = b.float().mean()
  print(accuracy)

0.6920334696769714
0.0
5.9604641222676946e-08
2.086161714487389e-07
6.854524485788716e-07
1.966946229003952e-06
1.4901158351676713e-07
0.0
0.0
2.9802320611338473e-08
tensor([0, 1, 1, 0]) tensor([[16.6574,  0.0000],
        [ 0.0000, 16.6624],
        [ 0.0000, 16.7689],
        [16.6715,  0.0000]])
tensor(1.)


In [ ]:
import torch
import torch.nn as nn
import csv

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1)

class CNN(nn.Module):
   def __init__(self):
    super().__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d()
    )

data = []

file = open('/content/sample_data/mnist_train_small.csv')
filereader = csv.reader(file)
for i in filereader:
  data.append(i)

# data = torch.tensor(data)
# data.shape
data = list(map(int, data))
type(data)

TypeError: ignored